In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
data = pd.read_csv('/kaggle/input/company-bankruptcy-prediction/data.csv')
data.head()

In [ ]:
data.info()

> * ## Based on above we can say that there are 95 features and out of these there are no Categrical features.  All features are either int or float

In [ ]:
## Plotting the correlation matrix

In [ ]:
data.corr()

In [ ]:
from matplotlib import pyplot as plt
import seaborn as sns
data.var()

In [ ]:
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import StandardScaler

In [ ]:
##split the data
X = data.drop(['Bankrupt?'], axis = 1)
y = data['Bankrupt?']
X.shape , y.shape

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=.3, 
                                                    stratify=y, 
                                                    random_state=30)
x_train.shape, x_test.shape, y_train.shape, y_test.shape

In [ ]:
pipe = Pipeline((
("pt",PowerTransformer()),
("pca",PCA(n_components = 30)),
("lr",LogisticRegression()),
))
pipe.fit(x_train,y_train)
print('Testing accuracy')
print(pipe.score(x_test, y_test))
print('TRaining accuracy')
print(pipe.score(x_train, y_train))

In [ ]:
test_data = []
train_data = []
for i in range(10,80):
    #print(i)
    pipe = Pipeline((
    ("pt",PowerTransformer()),
    ("pca",PCA(n_components = i)),
    ("lr",LogisticRegression()),
    ))
    pipe.fit(x_train,y_train)
    #print('Testing accuracy')
    test_data.append(pipe.score(x_test, y_test))
    #print('TRaining accuracy')
    train_data.append(pipe.score(x_train, y_train))
    

In [ ]:
from matplotlib import pyplot as plt
import seaborn as sns

In [ ]:
plt.figure(figsize=(12,6))
plt.plot(range(10,80), train_data,test_data, marker = "o" )
plt.title('Number of PCA components vs TRain and Test accuracy')
plt.xlabel('# of PCA component')
plt.ylabel('Accuracy')
plt.show()

In [ ]:
## Accuracy increases unchanged for components of PCA. We will use Component = 80

In [ ]:
## Now use decision Tree to check model performance

In [ ]:
from sklearn.tree import DecisionTreeClassifier
pipe = Pipeline((
("pt",PowerTransformer()),
("pca",PCA(n_components = 80)),
("lr",DecisionTreeClassifier(criterion='entropy',max_depth = 2)),
))

pipe.fit(x_train,y_train)
print('Testing accuracy')
print(pipe.score(x_test, y_test))
print('TRaining accuracy')
print(pipe.score(x_train, y_train))

## For decision tree Balanced A

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
from sklearn.metrics import balanced_accuracy_score

clf = DecisionTreeClassifier(criterion='entropy',max_depth = 10)
model = clf.fit(x_train, y_train)
#balanced_accuracy_score(y_test,y_pred_test)
print('Testing accuracy')
print(clf.score(x_test, y_test))
print('Training accuracy')
print(clf.score(x_train, y_train))
print('Balanced Test accuracy')
y_test_pred = clf.predict(x_test)
print(balanced_accuracy_score(y_test,y_test_pred))
print('Balanced Training accuracy')
y_train_pred = clf.predict(x_train)
print(balanced_accuracy_score(y_train,y_train_pred))

In [ ]:
fig = plt.figure(figsize=(25,20))
_ = tree.plot_tree(clf, 
#                   feature_names=iris.feature_names,  
#                   class_names=iris.target_names,
                   filled=True)

In [ ]:
fig.savefig("decistion_tree.png")

In [ ]:
## Balanced Decision Tree

In [ ]:
text_representation = tree.export_text(clf)
print(text_representation)

In [ ]:
## Using random forest classifier

In [ ]:
from sklearn.feature_selection import RFE
pipe = Pipeline((
("pt",PowerTransformer()),
("pca",PCA(n_components = 40)),
("fs",RFE(estimator = LogisticRegression(),n_features_to_select=3, step=1)),
))
pipe.fit(x_train,y_train)
print('Testing accuracy')
print(pipe.score(x_test, y_test))
print('Training accuracy')
print(pipe.score(x_train, y_train))